# CSV

In [ ]:
'''
import pandas as pd

#dados Selic

codigo_serie = '11'
data_inicio_selic = '04/01/2021'
data_fim_selic = '30/09/2024'

url_selic = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=csv&dataInicial={data_inicio_selic}&dataFinal={data_fim_selic}'

dados_selic = pd.read_csv(url_selic, sep=';', encoding='ISO-8859-1')

dados_selic = dados_selic.rename(columns={'valor': 'selic_ao_dia'})

dados_selic.set_index('data', inplace=True)

dados_selic.index = pd.to_datetime(pd.to_datetime(dados_selic.index,  format='mixed').date)

Dados_Selic_CSV = dados_selic.to_csv('Dados_Selic_CSV.csv')

'''

In [ ]:
'''# Gerar CSV atualizado com dados da CVM Sicredi Fia Petr

# dataframe com variação diária
import pandas as pd
import zipfile 
import requests

pd.options.display.float_format = '{:4f}'.format #para melhor visualização dos dados (casas decimais muito grandes)

# jan/21 até ago/24
anos = range(2021, 2025)  
meses = range(1, 13)  

anos_meses = [(ano, f"{mes:02d}") for ano in anos for mes in meses if (ano == 2024 and mes <= 9) or (ano < 2024)]

# Usar Nome associado ao CNPJ:

cadastro_fundos = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv',
                              sep = ';',
                              encoding = 'ISO-8859-1')

cadastro_fundos = cadastro_fundos[['CNPJ_FUNDO', 'DENOM_SOCIAL']]

cadastro_fundos = cadastro_fundos.drop_duplicates()

base_dados2 = pd.DataFrame()  # DF vazio 

for ano, mes in anos_meses:

    url = f"https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{ano}{mes}.zip"

    paginaweb = requests.get(url)

    with open(f"inf_diario_fi_{ano}{mes}.zip", 'wb') as arquivo_cvm:

        arquivo_cvm.write(paginaweb.content)

    arquivo_zip = zipfile.ZipFile(f"inf_diario_fi_{ano}{mes}.zip")

    dados_fundos = pd.read_csv(arquivo_zip.open(arquivo_zip.namelist()[0]), sep=';',    )

    # processar dados ordenados cronologicamente
    dados_fundos['DT_COMPTC'].sort_values(ascending=True)

    dados_fundos = pd.merge(dados_fundos, cadastro_fundos, how="left", left_on="CNPJ_FUNDO", right_on="CNPJ_FUNDO")

    dados_fundos = dados_fundos[['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_COMPTC', 'VL_QUOTA', 'VL_PATRIM_LIQ', 'NR_COTST']]

    # fundo SICREDI
    fundo_sicredi_petr = dados_fundos[dados_fundos['CNPJ_FUNDO'].str.contains('08.336.054/0001-34', na=False)]

    # Concatenando dados em um dataframe (base de dados)
    base_dados2 = pd.concat([base_dados2, fundo_sicredi_petr], ignore_index=True)

base_dados2.to_csv('Dados_Inicio_Fim_Dia_SICREDIPETR.csv', index=False)'''

In [ ]:
#dados do fundo SICREDI FIA PETROBRAS

import pandas as pd

dados_bruto_diario = pd.read_csv('Dados_Inicio_Fim_Dia_SICREDIPETR.csv')

dados_sicredi_cru = dados_bruto_diario[['DT_COMPTC', 'VL_QUOTA']]

dados_sicredi_cru['DT_COMPTC'] = pd.to_datetime(dados_sicredi_cru['DT_COMPTC'])

# Teste

In [7]:
import pandas as pd

dados_selic = pd.read_csv("C:\\Users\\Guima\\Desktop\\Códigos IC\\Dados_Selic_CSV.csv", decimal=',')

# Tratar dados - Data e Valor de str para float

dados_selic['selic_ao_dia'] = pd.to_numeric(dados_selic['selic_ao_dia'])

dados_selic.columns = ['Data'] + list(dados_selic.columns[1:])
'''''
from datetime import datetime

# Exemplo de DataFrame

def corrigir_data(data_str):
    try:
        # Tentar converter usando o formato %Y%d%m
        return datetime.strptime(data_str, "%Y%d%m").strftime("%Y-%m-%d")
    except ValueError:
        try:
            # Se falhar, verificar se está no formato padrão
            return pd.to_datetime(data_str).strftime("%Y-%m-%d")
        except:
            return data_str  # Retorna o valor original se não for possível converter

# Aplicar a função de correção ao DataFrame
dados_selic['Data_c'] = dados_selic['Data'].apply(corrigir_data)

'''

dados_selic['Data'] = dados_sicredi_cru['DT_COMPTC']

## DataFrame base - dados_sicredi


In [8]:
import pandas as pd
import numpy as np

# retornos diários 
dados_sicredi = dados_sicredi_cru[['DT_COMPTC', 'VL_QUOTA']]

dados_sicredi['Retornos_Diarios'] = dados_sicredi_cru['VL_QUOTA'].pct_change()

dados_sicredi = dados_sicredi[['DT_COMPTC', 'VL_QUOTA', 'Retornos_Diarios']]

dados_sicredi['Retornos_Diarios(%)'] = dados_sicredi['Retornos_Diarios']*100

dados_sicredi = dados_sicredi[['DT_COMPTC', 'VL_QUOTA', 'Retornos_Diarios(%)']]

# volatilidade

dados_sicredi['Volatilidade_Dia(%)'] = ((dados_sicredi['Retornos_Diarios(%)'].rolling(window=22)).std())*np.sqrt(252)

dados_sicredi = dados_sicredi[['DT_COMPTC', 'VL_QUOTA', 'Retornos_Diarios(%)', 'Volatilidade_Dia(%)']]

## Gráficos da Volatilidade, Retorno e Retorno Selic

In [ ]:
# gráfico volatilidade

import plotly.express as px

dados_sicredi_cru['DT_COMPTC'] = pd.to_datetime(dados_sicredi_cru['DT_COMPTC'])

dados_sicredi['DT_COMPTC'] = pd.to_datetime(dados_sicredi['DT_COMPTC'])

graph_vol = px.line(dados_sicredi, x='DT_COMPTC', y='Volatilidade_Dia(%)')

graph_vol.show()

In [ ]:
# gráfico retorno x selic

import plotly.express as px
import plotly.graph_objects as go

dados_sicredi['DT_COMPTC'] = pd.to_datetime(dados_sicredi['DT_COMPTC'])

graph_return = px.line(dados_sicredi, x='DT_COMPTC', y='Retornos_Diarios(%)')

graph_selic =  px.line(dados_selic, x=dados_sicredi['DT_COMPTC'], y='selic_ao_dia')

graph_return.show()
graph_selic.show()

## Ajustes + ln returns + Sharpe Diario

In [11]:
# Datas como Índice

dados_sicredi_cru.set_index('DT_COMPTC', inplace=True)
dados_sicredi.set_index('DT_COMPTC', inplace=True)

In [ ]:
# LN dos valores das cotas

dados_sicredi['ln_quotas'] = np.log(dados_sicredi['VL_QUOTA'])

# retornos com o log 

dados_sicredi['retorno_log(%)'] = np.log(dados_sicredi['VL_QUOTA']/dados_sicredi['VL_QUOTA'].shift(1))*100

dados_sicredi = dados_sicredi[['VL_QUOTA','ln_quotas','Retornos_Diarios(%)','Volatilidade_Dia(%)','retorno_log(%)']]

# inserir selic

combinado1 = pd.merge(dados_sicredi,dados_selic,how='outer',left_index=True,right_on='Data')

combinado1.set_index('Data',inplace=True)

combinado1

In [13]:
# Sharpe / Dia

combinado1['Sharpe/Dia'] = (combinado1['Retornos_Diarios(%)']-combinado1['selic_ao_dia'])/combinado1['Volatilidade_Dia(%)']
combinado1 = combinado1[['selic_ao_dia','VL_QUOTA', 'ln_quotas', 'Retornos_Diarios(%)', 'retorno_log(%)','Volatilidade_Dia(%)', 'Sharpe/Dia']]


## Janelas de Tempo para ano, 1 mês, 3 meses, 6 meses, 12 meses, 24 meses

### Retornos nas Janelas de Tempo

In [14]:
# Definir as datas de corte para cada janela de tempo
date_24M = combinado1.index[-1] - pd.DateOffset(months=24)
date_12M = combinado1.index[-1] - pd.DateOffset(months=12)
date_6M = combinado1.index[-1] - pd.DateOffset(months=6)
date_3M = combinado1.index[-1] - pd.DateOffset(months=3)
date_1M = combinado1.index[-1] - pd.DateOffset(months=1)

# Filtrar os dados por janela de tempo
dados_sicredi_24M = combinado1.loc[date_24M:]
dados_sicredi_12M = combinado1.loc[date_12M:]
dados_sicredi_6M = combinado1.loc[date_6M:]
dados_sicredi_3M = combinado1.loc[date_3M:]
dados_sicredi_1M = combinado1.loc[date_1M:]

##### 1M

In [ ]:
combinado1['Retorno_1M(%)'] = combinado1['VL_QUOTA'].pct_change(periods=21)*100 # 21 dias / mês

##### 3M

In [ ]:
combinado1['Retorno_3M(%)'] = combinado1['VL_QUOTA'].pct_change(periods=65)*100 # 65 dias / 03 meses

##### 6M

In [ ]:
combinado1['Retorno_6M(%)'] = combinado1['VL_QUOTA'].pct_change(periods=123)*100 # 123 dias / 06 meses

##### 12M

In [ ]:
combinado1['Retorno_12M(%)'] = combinado1['VL_QUOTA'].pct_change(periods=252)*100 # 252 dias / 12 meses

##### 24M

In [ ]:
combinado1['Retorno_24M(%)'] = combinado1['VL_QUOTA'].pct_change(periods=503)*100 # 503 dias / 24 meses

##### 36M

In [ ]:
combinado1['Retorno_36M(%)'] = combinado1['VL_QUOTA'].pct_change(periods=752)*100 # 752 dias / 36 meses

##### ao ano

In [ ]:
# retorno 1 ano

sicredi_retornoAD_anual = combinado1.resample('YE').last()

sicredi_retornoAD_anual['Retornos_Ano(%)'] = sicredi_retornoAD_anual['VL_QUOTA'].pct_change(periods=1)*100

sicredi_retornoAD_anual


#### Tentativa de usar Janelas de tempo

In [23]:
# retorno mensal (1Mês)

#sicredi_retornoAD_mensal = combinado1.resample('ME').last()

#sicredi_retornoAD_mensal['Retorno_Mensal(%)'] = sicredi_retornoAD_mensal['VL_QUOTA'].pct_change(periods=1)*100

#sicredi_retornoAD_mensal


##### 3M

In [24]:
# retorno 3 meses

#sicredi_retornoAD_3mes = combinado1.resample('ME').last()

#sicredi_retornoAD_3mes['Retorno_3M(%)'] = sicredi_retornoAD_3mes['VL_QUOTA'].pct_change(periods=3)*100

#sicredi_retornoAD_3mes

##### 6M

In [25]:
# retorno 6 meses

#sicredi_retornoAD_6mes = combinado1.resample('ME').last()

#sicredi_retornoAD_6mes['Retorno_6M(%)'] = sicredi_retornoAD_6mes['VL_QUOTA'].pct_change(periods=6)*100

#sicredi_retornoAD_6mes

##### 12M

In [26]:
# retorno 12 meses

#sicredi_retornoAD_12mes = combinado1.resample('ME').last()

#sicredi_retornoAD_12mes['Retorno_12M(%)'] = sicredi_retornoAD_12mes['VL_QUOTA'].pct_change(periods=12)*100

#sicredi_retornoAD_12mes

##### 24M

In [27]:
# retorno 24 meses

#sicredi_retornoAD_24mes = combinado1.resample('ME').last()

#sicredi_retornoAD_24mes['Retorno_24M(%)'] = sicredi_retornoAD_24mes['VL_QUOTA'].pct_change(periods=24)*100

#sicredi_retornoAD_24mes

In [28]:
# retorno 12 meses

#sicredi_retornoAD_36mes = combinado1.resample('ME').last()

#sicredi_retornoAD_36mes['Retorno_36M(%)'] = sicredi_retornoAD_36mes['VL_QUOTA'].pct_change(periods=36)*100

#sicredi_retornoAD_36mes

### Volatilidade nas janelas de tempo

##### Volatilidade ao mês

In [ ]:
combinado1['Volatilidade_Mensal(%)'] = ((combinado1['Retornos_Diarios(%)'].rolling(window=21)).std())*np.sqrt(252)

##### Volatilidade 12 Meses

In [ ]:
combinado1['Volatilidade_12M(%)'] = ((combinado1['Retornos_Diarios(%)'].rolling(window=252)).std())*np.sqrt(252)

##### Volatilidade 24 Meses

In [ ]:
combinado1['Volatilidade_24M(%)'] = ((combinado1['Retornos_Diarios(%)'].rolling(window=503)).std())*np.sqrt(252)

##### Volatilidade 36 Meses

In [ ]:
combinado1['Volatilidade_36M(%)'] = ((combinado1['Retornos_Diarios(%)'].rolling(window=752)).std())*np.sqrt(252)

##### Volatilidade ao Ano!!!!!!!!!!!!!

In [33]:
sicredi_retornoAD_anual['Volatilidade_Ano(%)'] = ((sicredi_retornoAD_anual['Retornos_Ano(%)'].rolling(window=2)).std())*np.sqrt(252)

### Indice Sharpe nas Janelas de tempo calculadas

In [ ]:
# Selic para diferentes períodos

#combinado1['selic_ao_mes'] = ((1 + combinado1['selic_ao_dia']/100)**21 - 1)*100
#combinado1['selic_12M'] = ((1+combinado1['selic_ao_dia']/100)**252 - 1)*100

combinado1['selic_ao_mes'] = (combinado1['selic_ao_dia'].rolling(21)).sum()

combinado1['selic_12M'] = (combinado1['selic_ao_dia'].rolling(252)).sum()

In [ ]:
combinado1.tail(252)

In [ ]:
# IS para 1 mês

combinado1['IS_1M'] = (combinado1['Retorno_1M(%)'] - combinado1['selic_ao_mes']) / combinado1['Volatilidade_Mensal(%)']

In [ ]:
combinado1

##### Tentativa de Gerar dias úteis no período especificado

In [ ]:
'''import pandas as pd
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday
from pandas.tseries.offsets import CustomBusinessDay

# Criação de um calendário com os principais feriados brasileiros
class BrazilHolidayCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('Ano Novo', month=1, day=1),
        Holiday('Carnaval', month=2, day=20),  # Exemplo para 2024
        Holiday('Sexta-feira Santa', month=3, day=29),  # Exemplo para 2024
        Holiday('Tiradentes', month=4, day=21),
        Holiday('Dia do Trabalho', month=5, day=1),
        Holiday('Corpus Christi', month=5, day=30),  # Exemplo para 2024
        Holiday('Independência do Brasil', month=9, day=7),
        Holiday('Nossa Senhora Aparecida', month=10, day=12),
        Holiday('Finados', month=11, day=2),
        Holiday('Proclamação da República', month=11, day=15),
        Holiday('Natal', month=12, day=25),
        # Adicione mais feriados locais/regionalizados aqui, se necessário
    ]

# Definindo um custom business day (dias úteis) com base no calendário de feriados brasileiros
brazil_bd = CustomBusinessDay(calendar=BrazilHolidayCalendar())

# Função para calcular o número de dias úteis em um período específico
def calculate_business_days(start_date, end_date):
    # Gera uma lista de dias úteis (exclui finais de semana e feriados automaticamente)
    business_days = pd.date_range(start=start_date, end=end_date, freq=brazil_bd)
    
    return len(business_days)

# Função principal para calcular dias úteis em diferentes períodos
def calculate_business_days_for_periods():
    periods = [1, 3, 6, 12, 24, 36, 48]  # Períodos em meses
    end_date = pd.Timestamp.today()  # Data final como a data atual
    results = {}

    # Calcula o número de dias úteis para cada período
    for months in periods:
        # Data inicial (subtraindo o número de meses)
        start_date = end_date - pd.DateOffset(months=months)
        business_days_count = calculate_business_days(start_date, end_date)
        results[f'{months}_months'] = business_days_count
    
    return results

# Exemplo de uso: calcular e exibir os dias úteis para os períodos especificados
business_days_results = calculate_business_days_for_periods()

# Exibe o número de dias úteis em cada período
for period, count in business_days_results.items():
    print(f'Número de dias úteis em {period}: {count}')'''
